# Setups

In [1]:
!pip install tensorflow transformers pandas scikit-learn spacy wordcloud gensim
!pip install matplotlib seaborn
!pip install line_profiler

     |████████████████████████████████| 2.5MB 14.0MB/s 
     |████████████████████████████████| 3.3MB 52.6MB/s 
     |████████████████████████████████| 901kB 59.3MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 


In [66]:
import re
import os
import sys
import logging
from pathlib import Path

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc

import matplotlib.pyplot as plt
%matplotlib inline

In [56]:
%%html
<style>
table {float:left}
</style>

In [68]:
logging.disable(logging.WARNING)
logging.basicConfig(level=logging.ERROR)

## Google Colab

In [3]:
def google_colab_info():
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
        print('and then re-execute this cell.')
    else:
        print(gpu_info)

    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
        print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
        print('re-execute this cell.')
    else:
        print('You are using a high-RAM runtime!')

In [4]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
    google_colab_info()
    
    DATA_PATH="/content/drive/MyDrive/data/jigsaw-toxic-comment-classification-challenge.zip"
    google.colab.drive.mount('/content/drive')
except:
    IN_GOOGLE_COLAB = False
    DATA_PATH = input("Enter the data archive path") 

Fri Jun 25 14:51:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Pandas

In [5]:
import pandas as pd
pd.options.display.max_colwidth = 1000

## TensorFlow

| TF_CPP_MIN_LOG_LEVEL | Description|          
| - |------------- | 
|0| Suppress all messages are logged (default behavior)|
|1 |Suppress INFO messages are not printed|
|2 |Suppress INFO and WARNING messages are not printed|
|3 |Suppress INFO, WARNING, and ERROR messages are not printed|



In [65]:
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Transformers

* [Logging](https://huggingface.co/transformers/main_classes/logging.html)

In [62]:
os.environ['TRANSFORMERS_VERBOSITY'] = "error"
import transformers
transformers.logging.set_verbosity(transformers.logging.ERROR)

from transformers import (
    PreTrainedModel,
    DistilBertTokenizerFast,
    TFDistilBertForSequenceClassification,
    TFTrainer,
    TFTrainingArguments
)

In [7]:
class SavePretrainedCallback(tf.keras.callbacks.Callback):
    # Hugging Face models have a save_pretrained() method that saves both the weights and the necessary
    # metadata to allow them to be loaded as a pretrained model in future. This is a simple Keras callback
    # that saves the model with this method after each epoch.
    def __init__(self, output_dir, **kwargs):
        super().__init__()
        self.output_dir = output_dir

    def on_epoch_end(self, epoch, logs=None):
        self.model.save_pretrained(self.output_dir)


In [45]:
class Runner:
    # ================================================================================
    # Class
    # ================================================================================
    USE_HF_TRAINER = False
    _model_name = 'distilbert-base-cased'
    _tokenizer = DistilBertTokenizerFast.from_pretrained(_model_name)

    # ================================================================================
    # Instance
    # ================================================================================
    # --------------------------------------------------------------------------------
    # Instance properties
    # --------------------------------------------------------------------------------
    @property
    def batch_size(self):
        assert self._batch_size > 0
        return self._batch_size

    @property
    def X(self):
        """Training DataSet"""
        return self._X

    @property
    def V(self):
        """Validation DataSet"""
        return self._V

    @property
    def model_name(self):
        """HuggingFace pretrained model name"""
        return self._model_name

    @property
    def model(self):
        """Model"""
        return self._model

    @property
    def learning_rate(self):
        return self._learning_rate

    @property
    def num_epochs(self):
        return self._num_epochs

    @property
    def tokenizer(self):
        """"""
        return self._tokenizer

    @property
    def max_sequence_length(self):
        """"""
        return self._max_sequence_length

    @property
    def trainer(self):
        """"""
        return self._trainer

    @property
    def output_directory(self):
        """Directory to save models, etc"""
        return self._output_directory

    # --------------------------------------------------------------------------------
    # Instance initialization
    # --------------------------------------------------------------------------------
    def __init__(
            self,
            training_data,
            training_label,
            validation_data,
            validation_label,
            max_sequence_length=256,
            batch_size=16,
            learning_rate=5e-5,
            num_epochs=3,
            output_directory="./output"
    ):
        # --------------------------------------------------------------------------------
        # Keras Model
        # --------------------------------------------------------------------------------
        assert learning_rate > 0.0
        self._learning_rate = learning_rate
        self._model = None

        assert num_epochs > 0
        self._num_epochs = num_epochs

        assert batch_size > 0
        self._batch_size = batch_size

        self._output_directory = output_directory
        Path(self.output_directory).mkdir(parents=True, exist_ok=True)

        # --------------------------------------------------------------------------------
        # HuggingFace
        # --------------------------------------------------------------------------------
        self._model = TFDistilBertForSequenceClassification.from_pretrained(self.model_name)
        self._trainer = None
        assert 128 <= max_sequence_length <= 512
        self._max_sequence_length = max_sequence_length

        # --------------------------------------------------------------------------------
        # TensorFlow DataSet
        # --------------------------------------------------------------------------------
        assert np.all(np.isin(training_label, [0, 1]))
        assert np.all(np.isin(validation_label, [0, 1]))
        self._X = tf.data.Dataset.from_tensor_slices((
            dict(self.tokenizer(
                training_data,
                truncation=True,
                padding=True,
                max_length=self.max_sequence_length,
                return_tensors="tf"
            )),
            training_label
        ))
        self._V = tf.data.Dataset.from_tensor_slices((
            dict(self.tokenizer(
                validation_data,
                truncation=True,
                padding=True,
                max_length=self.max_sequence_length,
                return_tensors="tf"
            )),
            validation_label
        ))


    # --------------------------------------------------------------------------------
    # Instance methods
    # --------------------------------------------------------------------------------
    def _hf_train(self):
        self._training_args = TFTrainingArguments(
            output_dir='./results',             # output directory
            num_train_epochs=3,                 # total number of training epochs
            per_device_train_batch_size=self.batch_size,     # batch size per device during training
            per_device_eval_batch_size=self.batch_size,      # batch size for evaluation
            warmup_steps=500,                   # number of warmup steps for learning rate scheduler
            weight_decay=0.01,                  # strength of weight decay
            logging_dir='./logs',               # directory for storing logs
            logging_steps=10,
        )

        # with self._training_args.strategy.scope():
        #     self._model = TFDistilBertForSequenceClassification.from_pretrained(self.model_name)

        self._trainer = TFTrainer(
            model=self.model,
            args=self._training_args,   # training arguments
            train_dataset=self.X,       # training dataset
            eval_dataset=self.V         # evaluation dataset
        )
        self.trainer.train()

    def _keras_train(self):
        optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        self.model.compile(
            optimizer=optimizer, 
            # loss=self.model.compute_loss,
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics = ["accuracy"]
        )
        self.model.summary()
        self.model.fit(
            self.X.shuffle(1000).batch(self.batch_size).prefetch(1),
            epochs=self.num_epochs,
            batch_size=self.batch_size,
            validation_data=self.V.shuffle(1000).batch(self.batch_size).prefetch(1),
            # callbacks=[SavePretrainedCallback(output_dir=self.output_directory)],
        )

    def train(self):
        if self.USE_HF_TRAINER:
            self._hf_train()
        else:
            self._keras_train()

    def evaluate(self, data, label):
        assert np.all(np.isin(label, [0, 1]))
        test_dataset = tf.data.Dataset.from_tensor_slices((
            dict(self.tokenizer(
                data,
                truncation=True,
                padding=True,
                max_length=self.max_sequence_length,
                return_tensors="tf"
            )),
            label
        ))
        evaluation = self.model.evaluate(
            test_dataset.shuffle(1000).batch(self.batch_size).prefetch(1)
        )
        return evaluation

    def predict(self, data):
        tokens = dict(self.tokenizer(
            data,
            truncation=True,
            padding=True,
            max_length=self.max_sequence_length,
            return_tensors="tf"
        ))
        logits = self.model.predict(tokens)["logits"]
        return tf.nn.softmax(logits)

    def save(self, path_to_dir=None):
        if path_to_dir is None or len(path_to_dir) == 0:
            path_to_dir = self.output_directory
        Path(path_to_dir).mkdir(parents=True, exist_ok=True)
        if self.USE_HF_TRAINER:
            self.trainer.save_model(path_to_dir)  
        else:
            self.model.save_pretrained(path_to_dir)

    def load(self, path_to_dir=None):
        if path_to_dir is None or len(path_to_dir) == 0:
            path_to_dir = self.output_directory
        if os.path.isdir(path_to_dir) and os.access(path_to_dir, os.R_OK):
            self._model = TFDistilBertForSequenceClassification.from_pretrained(path_to_dir)


---
# Data

First, upload data to 

In [32]:
!unzip -o $DATA_PATH
!unzip -o train.csv.zip
!unzip -o test.csv.zip
!unzip -o test_labels.csv.zip

Archive:  /content/drive/MyDrive/data/jigsaw-toxic-comment-classification-challenge.zip
  inflating: sample_submission.csv.zip  
  inflating: test.csv.zip            
  inflating: test_labels.csv.zip     
  inflating: train.csv.zip           
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  test_labels.csv.zip
  inflating: test_labels.csv         


In [10]:
raw_train = pd.read_csv("./train.csv")
raw_test_data = pd.read_csv("./test.csv")
raw_test_label = pd.read_csv("./test_labels.csv")
raw_test = pd.merge(raw_test_data, raw_test_label, left_on='id', right_on='id', how='inner')

## Training (Raw)

In [11]:
raw_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0


In [12]:
raw_train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
raw_train[raw_train['toxic'] > 0].head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,"Hey... what is it..\n@ | talk .\nWhat is it... an exclusive group of some WP TALIBANS...who are good at destroying, self-appointed purist who GANG UP any one who asks them questions abt their ANTI-SOCIAL and DESTRUCTIVE (non)-contribution at WP?\n\nAsk Sityush to clean up his behavior than issue me nonsensical warnings...",1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming back! Tosser.",1,0,0,0,0,0
42,001810bf8c45bf5f,"You are gay or antisemmitian? \n\nArchangel WHite Tiger\n\nMeow! Greetingshhh!\n\nUh, there are two ways, why you do erased my comment about WW2, that holocaust was brutally slaying of Jews and not gays/Gypsys/Slavs/anyone...\n\n1 - If you are anti-semitian, than shave your head bald and go to the skinhead meetings!\n\n2 - If you doubt words of the Bible, that homosexuality is a deadly sin, make a pentagram tatoo on your forehead go to the satanistic masses with your gay pals!\n\n3 - First and last warning, you fucking gay - I won't appreciate if any more nazi shwain would write in my page! I don't wish to talk to you anymore!\n\nBeware of the Dark Side!",1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


## Test (Raw)
The label value -1 is not clear. Remove the rows where value is -1.

> test_labels.csv - labels for the test data; value of -1 indicates it was not used for scoring

In [14]:
raw_test = raw_test[(raw_test['toxic'] > 0)]   # Removing rows where 'toxic' label > 0 is sufficicent

In [15]:
raw_test.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,6090.0,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000
mean,1.0,0.060263,0.595402,0.033662,0.548768,0.113793
std,0.0,0.237993,0.490854,0.180372,0.497657,0.317586
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.0,0.000000,1.000000,0.000000,1.000000,0.000000
75%,1.0,0.000000,1.000000,0.000000,1.000000,0.000000
max,1.0,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
raw_test.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
21,00091c35fa9d0465,"== Arabs are committing genocide in Iraq, but no protests in Europe. == \n\n May Europe also burn in hell.",1,0,0,0,0,0
48,0013fed3aeae76b7,DJ Robinson is gay as hell! he sucks his dick so much!!!!!,1,0,1,0,1,1
59,0017d4d47894af05,":Fuck off, you anti-semitic cunt. |",1,0,1,0,1,0
76,001d739c97bc2ae4,"How dare you vandalize that page about the HMS Beagle! Don't vandalize again, demon!",1,0,0,0,0,0
81,001eff4007dbb65b,"::No, he is an arrogant, self serving, immature idiot. Get it right.",1,0,1,0,1,0


---
# BERT Fine Tuning

* [Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification](https://github.com/huggingface/transformers/issues/5421#issuecomment-652582854)


```
Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
```

> This is expected, and tells you that you won't have good performance with your BertForSequenceClassification model before you fine-tune it slightly_smiling_face.



In [49]:
BATCH_SIZE = 32
MAX_SEQUENTH_LENGTH = 256

In [50]:
def run(category):
    print("--------------------------------------------------------------------------------")
    print(f"{category}")
    print("--------------------------------------------------------------------------------")

    data = raw_train['comment_text'].tolist()
    label = raw_train[category].tolist()
    
    train_data, validation_data, train_label, validation_label = train_test_split(
        data,
        label,
        test_size=.2,
        shuffle=True
    )
    runner = Runner(
        training_data=train_data,
        training_label=train_label,
        validation_data=validation_data,
        validation_label=validation_label,
        batch_size=BATCH_SIZE,
        max_sequence_length=MAX_SEQUENTH_LENGTH,
        output_directory="/content/drive/MyDrive/data/model_C{}_B{}_L{}".format(
            category.upper(), BATCH_SIZE, MAX_SEQUENTH_LENGTH
        )
    )
    
    runner.train()
    runner.save()
    runners[category] = runner

    test_data = raw_test['comment_text'].tolist()
    test_label = raw_test[category].tolist()
    evaluation = runner.evaluate(test_data, test_label)
    evaluations[category] = evaluation
    print(f"Evaluation: (accuracy):{evaluation}")

In [51]:
categories = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
runners = {}
evaluations = {}

for category in categories:
    run(category)


--------------------------------------------------------------------------------
toxic
--------------------------------------------------------------------------------


Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier', 'dropout_279', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it f

Model: "tf_distil_bert_for_sequence_classification_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  65190912  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  1538      
_________________________________________________________________
dropout_279 (Dropout)        multiple                  0         
Total params: 65,783,042
Trainable params: 65,783,042
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
3990/3990 [==============================] - 1152s 287ms/step - loss: 0.1049 - accuracy: 0.9595 - val_loss: 0.0959 - val_accuracy: 0.9621
Epoch 2/3
3990/3990 [==============================] - 1143

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier', 'dropout_299', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it f

Model: "tf_distil_bert_for_sequence_classification_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  65190912  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  1538      
_________________________________________________________________
dropout_299 (Dropout)        multiple                  0         
Total params: 65,783,042
Trainable params: 65,783,042
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
3990/3990 [==============================] - 1148s 286ms/step - loss: 0.0293 - accuracy: 0.9895 - val_loss: 0.0254 - val_accuracy: 0.9902
Epoch 2/3
3990/3990 [==============================] - 1139

...

Model: "tf_distil_bert_for_sequence_classification_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  65190912  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  1538      
_________________________________________________________________
dropout_339 (Dropout)        multiple                  0         
Total params: 65,783,042
Trainable params: 65,783,042
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
3990/3990 [==============================] - 1147s 286ms/step - loss: 0.0165 - accuracy: 0.9968 - val_loss: 0.0125 - val_accuracy: 0.9971
Epoch 2/3
3990/3990 [==============================] - 1138

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier', 'dropout_359', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it f

Model: "tf_distil_bert_for_sequence_classification_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  65190912  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  1538      
_________________________________________________________________
dropout_359 (Dropout)        multiple                  0         
Total params: 65,783,042
Trainable params: 65,783,042
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
3990/3990 [==============================] - 1144s 285ms/step - loss: 0.0768 - accuracy: 0.9703 - val_loss: 0.0667 - val_accuracy: 0.9725
Epoch 2/3
3990/3990 [==============================] - 1136

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_379']
You should probably TRAIN this model on a down-stream task to be able to use it f

Model: "tf_distil_bert_for_sequence_classification_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  65190912  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  1538      
_________________________________________________________________
dropout_379 (Dropout)        multiple                  0         
Total params: 65,783,042
Trainable params: 65,783,042
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
3990/3990 [==============================] - 1145s 285ms/step - loss: 0.0329 - accuracy: 0.9907 - val_loss: 0.0258 - val_accuracy: 0.9919
Epoch 2/3
3990/3990 [==============================] - 1136

In [52]:
test_data = raw_test['comment_text'].tolist()
categories = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

for category in categories:
    index = np.random.randint(0, len(test_data))
    data = test_data[index]

    prediction = runners[category].predict(data)
    print(f"category: {category} prediction: {prediction}\ndata: {data}")

category: toxic prediction: [[0.04939984 0.95060015]]
data: IM SO GAY I WATCH GFAY PEOPLE LICK THEMSLVES
category: severe_toxic prediction: [[0.99324554 0.00675443]]
data: If you don't know any better then shut up!!!
category: obscene prediction: [[0.18318702 0.816813  ]]
data: CAPTIAN UNDERPANTS SUCKS WHO WOULD THINK OF SUCH A DAMN STUPID IDEA
category: threat prediction: [[9.9929214e-01 7.0781034e-04]]
data: PLEASE ALL CALM DOWN !! LONDON WAS BOMBED.. AND I'll KILL THE PAKIES
category: insult prediction: [[0.7141192  0.28588083]]
data: " 

 ""Duur duuur duur ""I don't negotiate with terrorists""? What are you like five fckko? Watch a lil too much 24 did ya? Where's a terrorist? 
 What are you supposedly ""negotiating"" with retard? 

 Just so you all know there's too many ips and computers here to stop me from trashing wiki as I did over the last week. 
 Over 200 tine minor edits to numbers dates and measurements. 

 So keep pretending you're ""negotiating with terrorists"" you fckng

---

In [78]:
test_data = raw_test['comment_text'].tolist()
test_label = raw_test[category].tolist()

dummy_data = test_data[:5]
dummy_label = test_label[:5]

runner = Runner(
    training_data=dummy_data,
    training_label=dummy_label,
    validation_data=dummy_data,
    validation_label=dummy_label,
    batch_size=BATCH_SIZE,
    max_sequence_length=MAX_SEQUENTH_LENGTH,
)

path_to_dir = "/content/drive/MyDrive/data/model_CTOXIC_B16_L512"
runner.load(path_to_dir)

In [80]:
data = test_data[np.random.randint(0, len(test_data))]
print(f"prediction {np.argmax(runner.predict(data).numpy().tolist()[0])} for data [{data}]")


prediction 1 for data [== You == 

 Can I suck your cock?]
